# Imports

In [1]:
import numpy as np
import pandas as pd

# Load in Data

In [2]:
pickle_to_load = "../../FantasyData/data-frames/df_wr_college_2000_2023.pkl"

In [3]:
df_pos = pd.read_pickle(pickle_to_load)

# Clean Up Data

In [4]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

# Get Draft Capital Success Rate By Round

Grab the best (lower tier is better) tier a player has been in, for each round of the draft:

In [5]:
tier_counts = df_clean.groupby(['draft_round', 'player_name'])['tier'].min().reset_index()

Then group this data frame by the draft round to get the total number of players in each round

In [6]:
total_players_per_round = tier_counts \
    .groupby('draft_round')['player_name'] \
    .count() \
    .reset_index(name='total_players')

Grab the number of each players in each tier, by their draft round:

In [7]:
players_in_tier = tier_counts.groupby(['draft_round', 'tier']).size().reset_index(name='players_in_tier')

Merge the dataframes together:

In [8]:
merged_data = pd.merge(players_in_tier, total_players_per_round, on='draft_round')
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

Tier percentagers by draft round:

In [9]:
merged_data.head(5)

,draft_round,tier,players_in_tier,total_players,percentage
0,1.0,1,30,74,40.540541
1,1.0,2,11,74,14.864865
2,1.0,3,11,74,14.864865
3,1.0,4,4,74,5.405405
4,1.0,5,3,74,4.054054


# Analysis

In [10]:
df_success = merged_data.query("tier <= 4").groupby("draft_round").apply(lambda x: x)

/tmp/ipykernel_22177/3608773231.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success = merged_data.query("tier <= 4").groupby("draft_round").apply(lambda x: x)


In [11]:
df_success

draft_round  tier  players_in_tier  total_players  percentage
draft_round                                                                  
1.0         0           1.0     1               30             74   40.540541
            1           1.0     2               11             74   14.864865
            2           1.0     3               11             74   14.864865
            3           1.0     4                4             74    5.405405
2.0         11          2.0     1               21             85   24.705882
            12          2.0     2               12             85   14.117647
            13          2.0     3               10             85   11.764706
            14          2.0     4               11             85   12.941176
3.0         23          3.0     1               12             72   16.666667
            24          3.0     2                9             72   12.500000
            25          3.0     3                5             72    6.944444
            26          3.0     4                7             72    9.722222
4.0         38          4.0     1                3             64    4.687500
            39          4.0     2                4             64    6.250000
            40          4.0     3                7             64   10.937500
            41          4.0     4                2             64    3.125000
5.0         55          5.0     1                5             48   10.416667
            56          5.0     2                1             48    2.083333
            57          5.0     3                4             48    8.333333
            58          5.0     4                4             48    8.333333
6.0         72          6.0     1                2             55    3.636364
            73          6.0     4                5             55    9.090909
7.0         88          7.0     1                2             43    4.651163
            89          7.0     2                2             43    4.651163

In [12]:
df_bust = merged_data.query("tier > 4").groupby("draft_round").agg({"percentage": "sum"})
df_bust

,percentage
draft_round,
1.0,24.324324
2.0,36.470588
3.0,54.166667
4.0,75.000000
5.0,70.833333
6.0,87.272727
7.0,90.697674
